In [1]:
import os
os.chdir("C:/Users/daess/Desktop/study/공모전/2022 빅콘테스트/Data/Using Data")

In [2]:
import pandas as pd
import numpy as np

In [3]:
visit_jeju = pd.read_csv('비짓제주 전처리.csv')

In [4]:
visit_jeju.head()

,CL_NM,AREA_NM,ADDR,ALL_TOTAL_CO
0,관광지,1100고지(휴게소),서귀포시1100로1555,2679
1,관광지,1100고지습지,서귀포시색달동산1-2,3860
2,관광지,4·3해원방사탑,제주시신산로82,280
3,관광지,5.16도로숲터널,서귀포시남원읍신례리산2-1,2454
4,관광지,9.81파크,제주시애월읍천덕로880-24,3499


In [5]:
# 행정구역 단위가 가장 작은 면부터 찾기
sections = {'면' : {1 : '한경면', 2 : '안덕면', 3 : ['표선면', '우도면']},
            '읍' : {1 : ['한림읍', '애월읍'], 2 : '대정읍', 3 : ['남원읍', '성산읍'], 4 : ['조천읍', '구좌읍']},
            '시' : {1 : '제주시', 2 : '서귀포시'}}

In [6]:
# 구역 반환 함수
def get_section(addr):

    def myeon_section(addr):
        myeon = sections['면']
        for num, value in myeon.items():
            if isinstance(value, list): # 구역에 해당하는 면이 여러개일 경우
                for i in range(len(value)):
                    if value[i] in addr:
                        return num
            else:
                if value in addr: # 구역에 해당하는 면이 한개일 경우
                    return num
    
    def eup_section(addr):
        eup = sections['읍']
        for num, value in eup.items():
            if isinstance(value, list): # 구역에 해당하는 읍이 여러개일 경우
                for i in range(len(value)):
                    if value[i] in addr:
                        return num
            else:
                if value in addr: # 구역에 해당하는 면이 한개일 경우
                    return num

    def si_section(addr):
        si = sections['시']
        for num, value in si.items(): 
            if isinstance(value, list): # 구역에 해당하는 시가 여러개일 경우
                for i in range(len(value)):
                    if value[i] in addr:
                        return num
            else:
                if value in addr: # 구역에 해당하는 면이 한개일 경우
                    return num
    
    if '면' in addr:
        section_get = myeon_section(addr)

        if pd.isna(section_get):
        # 도로명에 면이 포함되어 처리가 안된 경우
            if '읍' in addr:
                section_get = eup_section(addr)
                if pd.isna(section_get):
                    if '시' in addr:
                        section_get = si_section(addr)
    
    elif '읍' in addr:
        section_get = eup_section(addr)

        if pd.isna(section_get):
        # 도로명에 '읍'이 있을 경우
            if '시' in addr:
                section_get = si_section(addr)

    else:
        section_get = si_section(addr)

    return section_get       

    

In [7]:
# 구역 찾기
visit_jeju['SECTION'] = 0 # 빈 '구역 변수' 열 만들기

for i in range(len(visit_jeju)):
    addr = visit_jeju.loc[i, 'ADDR']
    visit_jeju.loc[i, 'SECTION'] = get_section(addr)


In [8]:
# 구역 변수에 결측치가 있음을 확인할 수 있음
visit_jeju.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3563 entries, 0 to 3562
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   CL_NM         3563 non-null   object 
 1   AREA_NM       3563 non-null   object 
 2   ADDR          3563 non-null   object 
 3   ALL_TOTAL_CO  3563 non-null   int64  
 4   SECTION       3562 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 139.3+ KB


In [9]:
visit_jeju[visit_jeju['SECTION'].isnull()]

,CL_NM,AREA_NM,ADDR,ALL_TOTAL_CO,SECTION
441,관광지,성산초등학교서청특별중대주둔지,서귀포시성산면성산리171-9,277,NaN


In [10]:
# 성산초등학교서청특별중대주둔지의 경우 지도에 나오지 않는 곳이므로 삭제
# 성산면도 없는 행정구역
seongsan = visit_jeju[visit_jeju['ADDR'].str.contains('성산면')].index
visit_jeju.drop(seongsan, inplace = True)

In [11]:
# 결측치로 인해 SECTION 변수가 float형이 되어 다시 int형으로 변환
visit_jeju['SECTION'] = visit_jeju['SECTION'].astype('int')

In [12]:
visit_jeju

,CL_NM,AREA_NM,ADDR,ALL_TOTAL_CO,SECTION
0,관광지,1100고지(휴게소),서귀포시1100로1555,2679,2
1,관광지,1100고지습지,서귀포시색달동산1-2,3860,2
2,관광지,4·3해원방사탑,제주시신산로82,280,1
3,관광지,5.16도로숲터널,서귀포시남원읍신례리산2-1,2454,3
4,관광지,9.81파크,제주시애월읍천덕로880-24,3499,1
...,...,...,...,...,...
3558,음식점,흑돼지촌,제주시한림읍한림상로62,143,1
3559,음식점,흑돼지킹생구이전문점,제주시도령로11길26-9,99,1
3560,음식점,흑소랑,제주시연북로631,240,1
3561,음식점,흑염소마루,서귀포시남원읍태위로663번길1-1,136,3


In [13]:
# 각 구역별 장소종류별 총 장소 수
visit_jeju.groupby(['SECTION', 'CL_NM']).count()

AREA_NM  ADDR  ALL_TOTAL_CO
SECTION CL_NM                             
1       관광지        370   370           370
        쇼핑          62    62            62
        숙박         353   353           353
        음식점        787   787           787
2       관광지        274   274           274
        쇼핑          22    22            22
        숙박         252   252           252
        음식점        351   351           351
3       관광지        168   168           168
        쇼핑           8     8             8
        숙박         162   162           162
        음식점        219   219           219
4       관광지        161   161           161
        쇼핑          22    22            22
        숙박         149   149           149
        음식점        202   202           202

In [14]:
SECTION_1 = visit_jeju[visit_jeju.SECTION == 1]
SECTION_2 = visit_jeju[visit_jeju.SECTION == 2]
SECTION_3 = visit_jeju[visit_jeju.SECTION == 3]
SECTION_4 = visit_jeju[visit_jeju.SECTION == 4]

In [16]:
visit_fix = pd.read_csv('visit_jeju_fix_.csv', encoding = 'ANSI')

In [20]:
visit_fix

,CL_NM,AREA_NM,ADDR,LTNO_ADDR,USE_MONTH,ALL_TOTAL_CO,SUM_GRGS_DSAMT
0,관광지,1100고지(휴게소),서귀포시1100로1555,서귀포시색달동1-3번지,11,2679,276.610821
1,관광지,1112도로,제주시명림로584,제주시봉개동78-1번지,3,6206,3226.512929
2,관광지,4·3해원방사탑,제주시신산로82,제주시일도이동837-20번지,9,280,306.031221
3,관광지,WE호텔웰니스센터,서귀포시1100로453-95,서귀포시회수동30번지,3,1658,4044.647019
4,관광지,간세공방,서귀포시중정로22,서귀포시서귀동316-1번지,9,999,209.579555
...,...,...,...,...,...,...,...
830,음식점,흑돼지구이집하영,서귀포시천제연로101,서귀포시색달동2477-5번지,9,565,207.232761
831,음식점,흑돼지돈까스뷔페밀라네사,제주시매촌2길78,제주시도련일동1751-7번지,9,207,326.499613
832,음식점,흑돼지킹생구이전문점,제주시도령로11길26-9,제주시연동2313-4번지,9,99,72.424229
833,음식점,흑소랑,제주시연북로631,제주시아라이동3007-2번지,9,240,209.952555
